### Importing necessary libraries...

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report

df_synth = pd.read_csv("data/PS_20174392719_1491204439457_log.csv")

In [3]:
df_synth.columns

Index(['step', 'type', 'amount', 'nameOrig', 'oldbalanceOrg', 'newbalanceOrig',
       'nameDest', 'oldbalanceDest', 'newbalanceDest', 'isFraud',
       'isFlaggedFraud'],
      dtype='object')

In [11]:
df_synth.type

0           PAYMENT
1           PAYMENT
2          TRANSFER
3          CASH_OUT
4           PAYMENT
             ...   
6362615    CASH_OUT
6362616    TRANSFER
6362617    CASH_OUT
6362618    TRANSFER
6362619    CASH_OUT
Name: type, Length: 6362620, dtype: object

#### Filtering to CASH_OUT and TRANSFER and building features

In [15]:
df_synth_ct = df_synth[df_synth["type"].isin(["CASH_OUT", "TRANSFER"])].copy()
df_synth_ct["type_code"] = (df_synth_ct["type"] == "TRANSFER").astype(int)

features = ["amount", "oldbalanceOrg", "newbalanceOrig", "type_code"]
X = df_synth_ct[features]
Y = df_synth_ct["isFraud"]

Y.value_counts(normalize=True)

isFraud
0    0.997035
1    0.002965
Name: proportion, dtype: float64

## Train/test split and logistic regression

In [22]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42, stratify=Y)

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, Y_train)

Y_pred = clf.predict(X_test)

print(confusion_matrix(Y_test, Y_pred))
print(classification_report(Y_test, Y_pred, digits=4))

[[552381     58]
 [   958    685]]
              precision    recall  f1-score   support

           0     0.9983    0.9999    0.9991    552439
           1     0.9219    0.4169    0.5742      1643

    accuracy                         0.9982    554082
   macro avg     0.9601    0.7084    0.7866    554082
weighted avg     0.9980    0.9982    0.9978    554082



#### Interpretation of the Confusion Matrix:

- True negatives (TN, legit correctly flagged legit): 552381.​

- False positives (FP, legit flagged as fraud): 58.​

- False negatives (FN, fraud missed): 958.​

- True positives (TP, fraud correctly flagged): 685.​

- So out of 1643 frauds, the model catches 685 and misses 958.

#### The metrics further tell us:

- Fraud precision ≈ 0.92: when the model says “fraud”, it is right 92% of the time, which is excellent and means few false alarms relative to the number of alerts.​

- Fraud recall ≈ 0.42: the model only catches about 42% of frauds, so it still misses most fraudulent transactions.​

- Overall accuracy ≈ 99.8% is not very informative here because the dataset is extremely imbalanced and we could get similar accuracy by predicting “non-fraud” almost always.

#### So, for AML, this recall is usually too low, because missing frauds is very costly, so our next steps are typically to adjust the decision threshold, reweight classes, or use a different model to trade some precision for higher recall.